In [ ]:
import sys
from pathlib import Path
from os import getcwd
sys.path.append(str(Path(getcwd()).parent.parent))

In [ ]:
import pandas as pd
import numpy as np

from datetime import datetime, date, timedelta

today = pd.to_datetime(date.today())

In [ ]:

from bulletin import default_input, default_output
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.notifica import Notifica
from bulletin.systems.esus import eSUS

In [ ]:
cc = CasosConfirmados(f"cc_{today.strftime('%d_%m_%Y')}")
cc.load()
cc.df.shape

In [ ]:
notifica = Notifica('notifica')
notifica.load(compress=False)
notifica.df.shape

In [ ]:
vacinados_esus = eSUS('vacinados')
vacinados_esus.load()
vacinados_esus.df.shape

In [ ]:
faixa_etaria = [60,65,70,75,80,85,90]
faixa_etaria_labels = ['0-59','60-64','65-69','70-74','75-79','80-84','85-89','>=90']

vacinados = vacinados_esus.df

casos_confirmados = cc.df.rename(columns={'id_notifica':'id'})
casos_confirmados['faixa_etaria'] = [ faixa_etaria_labels[idx] for idx in np.digitize(casos_confirmados['idade'],faixa_etaria,right=False)]
casos_confirmados['ano_caso'] = casos_confirmados['data_diagnostico'].apply(lambda x: x.year if isinstance(x,date) else 'erro')
casos_confirmados['mes_caso'] = casos_confirmados['data_diagnostico'].apply(lambda x: x.month if isinstance(x,date) else 'erro')
casos_confirmados['ano_obito'] = casos_confirmados['data_cura_obito'].apply(lambda x: x.year if isinstance(x,date) else 'erro')
casos_confirmados['mes_obito'] = casos_confirmados['data_cura_obito'].apply(lambda x: x.month if isinstance(x,date) else 'erro')

casos_vacinados_cns = pd.merge(vacinados,casos_confirmados[['cns','id','data_diagnostico','evolucao','data_cura_obito']], on='cns', how='inner')
print(len(casos_vacinados_cns))

casos_vacinados_cpf = pd.merge(vacinados.loc[vacinados['cpf'].notna()],casos_confirmados[['cpf','id','data_diagnostico','evolucao','data_cura_obito']], on='cpf', how='inner')
print(len(casos_vacinados_cpf))

casos_vacinados_mae = pd.merge(vacinados.loc[vacinados['hash_mae'].notna()],casos_confirmados[['hash_mae','id','data_diagnostico','evolucao','data_cura_obito']], on='hash_mae', how='inner')
print(len(casos_vacinados_mae))

casos_vacinados_nasc = pd.merge(vacinados.loc[vacinados['hash_nasc'].notna()],casos_confirmados[['hash_nasc','id','data_diagnostico','evolucao','data_cura_obito']], on='hash_nasc', how='inner')
print(len(casos_vacinados_nasc))

casos_vacinados = pd.concat([casos_vacinados_cns,casos_vacinados_cpf,casos_vacinados_mae,casos_vacinados_nasc])
casos_vacinados = casos_vacinados.drop_duplicates('id', keep='first')
print(len(casos_vacinados))

casos_vacinados[['id','evolucao']].groupby('evolucao').count()

casos_vacinados['dias_entre_dose_diagnostico'] = 0
'''
se 
    a data de diagnóstico for anterior a vacinação
entao -1
'''
casos_vacinados.loc[(
    (casos_vacinados['situacao'] == 'inconsistencia') |
    (casos_vacinados['data_aplicacao_1a_dose'] > casos_vacinados['data_diagnostico'])
), 'dias_entre_dose_diagnostico'] = -1

'''
se 
    só a primeira dose foi aplicada
então    
    dias_entre_dose_diagnostico será data_diagnostico - data_aplicacao_1a_dose
'''
casos_vacinados.loc[(
    (casos_vacinados['situacao'] == 'só primeira dose') &
    (casos_vacinados['data_aplicacao_1a_dose'] <= casos_vacinados['data_diagnostico'])
), 'dias_entre_dose_diagnostico'] = (casos_vacinados.loc[(
    (casos_vacinados['situacao'] == 'só primeira dose') &
    (casos_vacinados['data_aplicacao_1a_dose'] <= casos_vacinados['data_diagnostico'])
), 'data_diagnostico'] - casos_vacinados.loc[(
    (casos_vacinados['situacao'] == 'só primeira dose') &
    (casos_vacinados['data_aplicacao_1a_dose'] <= casos_vacinados['data_diagnostico'])
), 'data_aplicacao_1a_dose']).dt.days

'''
se 
    ambas as doses foram aplicadas
    e a data de diagnóstico está entre as doses 
então    
    dias_entre_dose_diagnostico será data_diagnostico - data_aplicacao_1a_dose
'''
casos_vacinados.loc[(
    (casos_vacinados['situacao'] == 'ambas doses aplicadas') &
    (casos_vacinados['data_aplicacao_1a_dose'] <= casos_vacinados['data_diagnostico']) &
    (casos_vacinados['data_aplicacao_2a_dose'] >= casos_vacinados['data_diagnostico'])
),'dias_entre_dose_diagnostico'] = (casos_vacinados.loc[(
    (casos_vacinados['situacao'] == 'ambas doses aplicadas') &
    (casos_vacinados['data_aplicacao_1a_dose'] <= casos_vacinados['data_diagnostico']) &
    (casos_vacinados['data_aplicacao_2a_dose'] >= casos_vacinados['data_diagnostico'])
),'data_diagnostico'] - casos_vacinados.loc[(
    (casos_vacinados['situacao'] == 'ambas doses aplicadas') &
    (casos_vacinados['data_aplicacao_1a_dose'] <= casos_vacinados['data_diagnostico']) &
    (casos_vacinados['data_aplicacao_2a_dose'] >= casos_vacinados['data_diagnostico'])
),'data_aplicacao_1a_dose']).dt.days

'''
se 
    ambas as doses foram aplicadas
    e a data de diagnóstico está depois da 2 dose
então    
    os dias entre dose e diagnóstico será data_diagnostico - data_aplicacao_2a_dose
'''
casos_vacinados.loc[(
    (casos_vacinados['situacao'] == 'ambas doses aplicadas') &
    (casos_vacinados['data_aplicacao_2a_dose'] <= casos_vacinados['data_diagnostico'])
),'dias_entre_dose_diagnostico'] = (casos_vacinados.loc[(
    (casos_vacinados['situacao'] == 'ambas doses aplicadas') &
    (casos_vacinados['data_aplicacao_2a_dose'] <= casos_vacinados['data_diagnostico'])
),'data_diagnostico'] - casos_vacinados.loc[(
    (casos_vacinados['situacao'] == 'ambas doses aplicadas') &
    (casos_vacinados['data_aplicacao_2a_dose'] <= casos_vacinados['data_diagnostico'])
),'data_aplicacao_2a_dose']).dt.days

casos_vacinados['vacinado'] = 0

casos_vacinados.loc[
    (casos_vacinados['situacao']=='ambas doses aplicadas') & 
    (casos_vacinados['data_diagnostico'] > (casos_vacinados['data_aplicacao_2a_dose'] + timedelta(14)) )
,'vacinado'] = 2

casos_vacinados.loc[
    (
        (casos_vacinados['situacao']=='ambas doses aplicadas') & 
        (casos_vacinados['data_diagnostico'] <= casos_vacinados['data_aplicacao_2a_dose'] ) &
        (casos_vacinados['data_diagnostico'] >= casos_vacinados['data_aplicacao_1a_dose'] )
    ) | (
        (casos_vacinados['situacao']=='só primeira dose') & 
        (casos_vacinados['data_diagnostico'] >= casos_vacinados['data_aplicacao_1a_dose'] )
    )
,'vacinado'] = 1

cc.df = pd.merge(casos_confirmados,casos_vacinados[['id','categoria','grupo_atendimento','fabricante','vacina_nome','situacao','vacinado']],how='left',on='id')

In [ ]:
cc.df['vacinado'] = cc.df['vacinado'].fillna(0)

In [ ]:
cc.df = cc.df.rename(columns={'id':'id_notifica'})

In [ ]:
gestantes = Notifica()
gestantes.df = pd.merge(cc.df,notifica.df.loc[notifica.df['gestante']==1, ['id','gestante_alto_risco']].rename(columns={'id':'id_notifica'}), on='id_notifica', how='left')
gestantes.toggle_merge_table('periodo_gestacao')
gestantes.df.loc[gestantes.df['gestante_alto_risco'].isin([0,3]), 'gestante_alto_risco'] = 'Não'
gestantes.df.loc[gestantes.df['gestante_alto_risco']==1, 'gestante_alto_risco'] = 'Sim'
gestantes.df.loc[gestantes.df['gestante_alto_risco']==2, 'gestante_alto_risco'] = 'Não'
gestantes.df

In [ ]:
gestantes.df = gestantes.df.loc[gestantes.df['gestante']==1]

In [ ]:
gestantes.df.groupby(['ano_caso','mes_caso','vacinado'])['vacinado'].count().unstack(-1).fillna(0).sum()

In [ ]:
gestantes.df.groupby(['ano_caso','mes_caso','vacinado'])['vacinado'].count().unstack(-1).fillna(0).plot()

In [ ]:
gestantes.df.groupby(['ano_caso','mes_caso','gestante_alto_risco'])['gestante_alto_risco'].count().unstack(-1).fillna(0).plot()